In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seml
import pandas as pd
import json
from collections import defaultdict

/nfs/homedirs/fuchsgru/miniconda3/lib/python3.9/site-packages/seml/database.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [83]:
collection_name = 'week6_partitions_and_purity'
collection = seml.database.get_collection(collection_name)
results = [{'config' : r['config'], 'result' : r['result'], 'id' : r['_id']} for r in collection.find() if r['status'] in ('COMPLETED',)]

In [84]:
len(results)

12

In [85]:
for result in results:
    with open(result['result']) as f:
        result['result'] = json.load(f)

In [89]:
data = []
for result in results:
    base = {
        'residual' : result['config']['model']['residual'],
        'train-labels' : tuple(result['config']['data']['train_labels']),
        'remove-vertices' : result['config']['data']['train_labels_remove_other'],
        'spectral-norm' : result['config']['model']['use_spectral_norm'],
    }
    for split_idx in range(result['config']['data']['num_dataset_splits']):
        for model_idx in range(result['config']['model']['num_initializations']):
            metrics = {k : v[result['config']['model']['num_initializations'] * split_idx + model_idx] for k, v in result['result'].items()}
            data.append({
                **base,
                **metrics,
                **{'split' : split_idx},
                **{'init' : model_idx},
            })
            
    
df = pd.DataFrame(data).groupby(['residual', 'train-labels', 'remove-vertices', 'spectral-norm', 'split', 'init']).mean()
df.head(50)
df.loc[True, (0, 1, 3, 4, 5), False, False]
#df.set_index(['train_labels', 'remove_vertices', 'residual', 'spectral_norm']).sort_index()

val_loss-val-train-labels  val_accuracy-val-train-labels  \
split init                                                             
0     0                      0.474701                           0.84   
      1                      0.458916                           0.84   
      2                      0.433209                           0.83   
      3                      0.456482                           0.83   
      4                      0.514577                           0.80   
1     0                      0.708936                           0.76   
      1                      0.684159                           0.75   
      2                      0.696963                           0.74   
      3                      0.723727                           0.74   
      4                      0.786649                           0.75   
2     0                      0.547628                           0.84   
      1                      0.520969                           0.83   
      2                      0.550905                           0.82   
      3                      0.519997                           0.85   
      4                      0.598349                           0.84   
3     0                      0.610938                           0.81   
      1                      0.574742                           0.81   
      2                      0.582279                           0.79   
      3                      0.597909                           0.81   
      4                      0.609030                           0.78   
4     0                      0.557144                           0.77   
      1                      0.537599                           0.77   
      2                      0.521287                           0.81   
      3                      0.551673                           0.80   
      4                      0.555007                           0.77   

            val_loss-val-reduced  val_accuracy-val-reduced  \
split init                                                   
0     0                 0.474701                      0.84   
      1                 0.458916                      0.84   
      2                 0.433209                      0.83   
      3                 0.456482                      0.83   
      4                 0.514577                      0.80   
1     0                 0.708936                      0.76   
      1                 0.684159                      0.75   
      2                 0.696963                      0.74   
      3                 0.723727                      0.74   
      4                 0.786649                      0.75   
2     0                 0.547628                      0.84   
      1                 0.520969                      0.83   
      2                 0.550905                      0.82   
      3                 0.519997                      0.85   
      4                 0.598349                      0.84   
3     0                 0.610938                      0.81   
      1                 0.574742                      0.81   
      2                 0.582279                      0.79   
      3                 0.597909                      0.81   
      4                 0.609030                      0.78   
4     0                 0.557144                      0.77   
      1                 0.537599                      0.77   
      2                 0.521287                      0.81   
      3                 0.551673                      0.80   
      4                 0.555007                      0.77   

            auroc_softmax_entropy  auroc:gpc-full:no  auroc:2-mog:no  \
split init                                                             
0     0                  0.830169           0.589662        0.636076   
      1                  0.835443           0.868143        0.492616   
      2                  0.848101           0.823840        0.547468   
      3                  0.85970

In [57]:
result

{'config': {'overwrite': 1,
  'db_collection': 'week6_partitions_and_purity',
  'data': {'dataset': 'cora_ml',
   'num_dataset_splits': 5,
   'split_type': 'uniform',
   'test_portion': 0.6,
   'test_portion_fixed': 0.2,
   'train_labels': [0, 1, 3, 4, 5, 6],
   'train_labels_remove_other': True,
   'train_portion': 20,
   'val_labels': 'all',
   'val_portion': 20},
  'evaluation': {'ignore_exceptions': False,
   'pipeline': [{'type': 'EvaluateEmpircalLowerLipschitzBounds',
     'num_perturbations': 20,
     'min_perturbation': 0.1,
     'max_perturbation': 10.0,
     'num_perturbations_per_sample': 5,
     'seed': 1337,
     'perturbation_type': 'noise',
     'name': 'noise-perturbations'},
    {'type': 'EvaluateEmpircalLowerLipschitzBounds',
     'num_perturbations': 10,
     'min_perturbation': 0.1,
     'max_perturbation': 10.0,
     'num_perturbations_per_sample': 2,
     'seed': 1337,
     'perturbation_type': 'derangement',
     'name': 'derangement-perturbations'},
    {'type':